In [12]:
import os
import pandas as pd
from supabase import create_client, Client

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [13]:
import geopandas as gpd
from shapely.geometry import Point



response = supabase.table('entrances').select("*").execute()
data,_ = response

entrances_df = pd.DataFrame(data[1])
# Create a new column in your DataFrame for the geographic data
entrances_df['geometry'] = [Point(xy) for xy in zip(entrances_df['longitude'], entrances_df['latitude'])]

# Convert the DataFrame to a GeoDataFrame
entrances_gdf = gpd.GeoDataFrame(entrances_df, geometry='geometry')
# Set the coordinate reference system (CRS) to EPSG:4326 (WGS84)
entrances_gdf.crs = "EPSG:4326"

entrances_gdf

,entrance_id,longitude,latitude,entrance_destination,entrance_name,geometry
0,3308608988,101.712717,3.158762,None,None,POINT (101.71272 3.15876)
1,3308608989,101.712507,3.158809,None,None,POINT (101.71251 3.15881)
2,4092013971,101.614130,3.022231,None,None,POINT (101.61413 3.02223)
3,4400847336,101.698939,3.139210,None,A,POINT (101.69894 3.13921)
4,4952299503,101.732720,3.104671,None,B,POINT (101.73272 3.10467)
...,...,...,...,...,...,...
284,12076281187,101.681477,3.113727,None,None,POINT (101.68148 3.11373)
285,12076314945,101.704860,3.098311,None,None,POINT (101.70486 3.09831)
286,12084946657,101.710926,3.076792,None,None,POINT (101.71093 3.07679)
287,12084946658,101.711568,3.075897,None,None,POINT (101.71157 3.07590)


In [52]:
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import os
#Personal api_key stored in a config.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key
ors_key: str = os.environ.get('ORS_KEY')


ors_client = client.Client(key='ors_key') 

In [55]:
entrances_gdf.iloc[0]['geometry'].coords[0][0]

101.7127175

In [57]:
[entrances_gdf.iloc[0]['geometry'].coords[0][0],entrances_gdf.iloc[0]['geometry'].coords[0][1]]

[101.7127175, 3.1587619]

In [61]:
isochrone_params = {
              'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
isochrone_params['locations'] = [[entrances_gdf.iloc[0]['geometry'].coords[0][0],entrances_gdf.iloc[0]['geometry'].coords[0][1]]]

temp_iso = ors_client.isochrones(**isochrone_params)


In [44]:
def isoGeoJsonRetriever(parameters,stations,client):

    #ORS isochrones API takes input list of coordinates in field called location. This line creates that column
    stations['locations']  = stations.apply(lambda row: list([row.loc["longitude"],row.loc["latitude"]]) , axis = 1)
    iso_list = []

    for index, row in stations.iterrows():
        print("Retrieving Isochrone of {} station".format(stations.loc[index,'name']))
        parameters['locations'] = [row.loc['locations']]
    
        try:
            temp_iso = client.isochrones(**parameters)
            temp_iso = json.dumps(temp_iso)
            iso_list.append(temp_iso)
            print("Success")
        except Exception as e:
            print(f"Failed to retrieve isochrone for station {stations.loc[index,'name']}: {e}")
            iso_list.append(None)  # or some other default value
    
    stations['iso'] = iso_list

    return
